# Working with Quantum Circuits inCirq

This notebook is a tutorial on manipulating quantum circuits with the built-in funcionality provided by Google's Cirq. By "manipulating quantum circuits," we mean operations on quantum circuit of the following form:

* Inserting instructions in a circuit

* Simplifying a circuit



Let's first import Cirq so we have all of it's functionality.

In [2]:
import cirq

A quantum circuit is a list of instructions acting on qubits. In the cells below, we'll look at how to declare qubits and circuits using Cirq. Later on, we'll transition into more advanced and useful operations that Cirq allows users to perform on quantum circuits.

## Declaring Qubits in Cirq

To work with a quantum circuit, we need qubits to operate on. Cirq is heavily focused on near-term applications of quantum computers. Because near-term quantum computers typically have qubits arranged in a line or a two-dimensional grid, Cirq provides ```LineQubits``` as well as ```GridQubits```.

Let's first look at ```LineQubit```s and how to declare them.

In [6]:
# number of qubits
n = 10

# get a list of qubits with linear indices
qubits = [cirq.LineQubit(k) for k in range(n)]

The class ```LineQubit``` allows the user to get the index of the qubit with the instance ```LineQubit.x``` as shown below.

In [12]:
print(qubits[3].x)

3


The function ```LineQubit.is_adjacent``` is also a useful feature.

In [26]:
# qubit 3 is adjacent to qubit 4 on linear connectivity
print(qubits[3].is_adjacent(qubits[4]))

# qubit 3 is NOT adjacent to qubit 6 on linear connectivity
print(qubits[3].is_adjacent(qubits[6]))

# qubit 0 is NOT adjacent to qubit 0 on linear connectivity
print(qubits[0].is_adjacent(qubits[9]))

True
False
False


In [23]:
qubits[9].range(5)

[LineQubit(0), LineQubit(1), LineQubit(2), LineQubit(3), LineQubit(4)]

## Declaring Circuits

In [27]:
# generic circuit
circ = cirq.Circuit()

# circuit for a device
circ_device = cirq.Circuit(device=cirq.google.Bristlecone)


## Working with Circuits

### Methods Returning Circuit Information

There are many methods that return some information about the circuit. These include:

1. ```next_moment_operating_on(qubits)```
    * Finds the index of the next moment that touches the given qubits.
    
1. ```prev_moment_operationg_on(qubits)```
    * Finds the index of the previous **(says "next" in code docs --> typo!)** moment that touches the given qubits.
    
1. ```operation_at(qubit, moment_index)```
    * Finds the operation on a qubit within a moment indexed by ```moment_index```, if any.
    
1. ```all_qubits()```
    * Returns the qubits acted upon by operations in this circuit.
    
1. ```findall_operations(predicate)```
    * Find the locations of all operations that satisfy a given condition. Returns an iterator of (index, operation) tuples where each operation satisfies the ```predicate```.
    
1. ```to_unitary_matrix()```
    * Converts the circuit into a unitary matrix, if possible.

1. ```apply_unitary_effect_to_state()```
    * Left-multiplies a state vector by the circuit's unitary effect.
    
1. ```to_text_diagram()```
    * Returns text containing a diagram describing the circuit.
    
1. ```to_text_diagram_drawer()``` **(maybe omit)**
    * Returns a TextDiagramDrawer with the circuit drawn into it.

In the above, we have given a brief description of these functions and omitted most optional arguments. For complete information on these functions, please see the documentation or source code.

### Methods for Manipulating Circuits

The functions we included above are useful for getting circuit information, but they don't affect the circuit. Cirq contains many useful built-in functions for manipulating, or mutating, quantum circuits. We'll now look at these through a series of examples.

First we'll write a function for giving us a (random) circuit to work with.

In [134]:
import numpy as np

# one qubit operations dictionary
oneq_ops = {1 : cirq.X,
            2 : cirq.Y,
            3 : cirq.Z,
            4 : cirq.H,
            5 : cirq.X ** 0.5,
            6 : cirq.T}

# two qubit operations dictionary

def random_circuit(num_qubits, depth, 
                   oneq_ops_dict=oneq_ops):
    # get some qubits and a circuit
    qbits = [cirq.LineQubit(x) for x in range(num_qubits)]
    circ = cirq.Circuit()
    
    for _ in range(depth):
        op_keys = np.random.randint(1, len(oneq_ops) + 1, num_qubits)
        circ.append(
            [oneq_ops[key](q) for (q, key) in enumerate(op_keys)],
            strategy=cirq.InsertStrategy.EARLIEST
            )
        
    return circ

This function, when called, produces something like the following:

In [151]:
mycirc = random_circuit(3, 9)
print(mycirc)

0: ───X───X^0.5───T───X^0.5───T───────X^0.5───T───────X^0.5───Z───

1: ───Z───H───────X───X───────X^0.5───X^0.5───Z───────Z───────Z───

2: ───T───X───────H───T───────X^0.5───Y───────X^0.5───H───────H───


#### Circuit Slicing and Arithmetic

Circuits are built up of ```moment```s, or a set of operations that acts at a given time (i.e., moment). We can think of these as vertical slices of the circuit diagram. We can loop throug moments of the circuit as follows:

In [152]:
for moment in mycirc:
    print(moment)

X(0) and Z(1) and T(2)
X**0.5(0) and H(1) and X(2)
T(0) and X(1) and H(2)
X**0.5(0) and X(1) and T(2)
T(0) and X**0.5(1) and X**0.5(2)
X**0.5(0) and X**0.5(1) and Y(2)
T(0) and Z(1) and X**0.5(2)
X**0.5(0) and Z(1) and H(2)
Z(0) and Z(1) and H(2)


In addition, we can slice circuits to see particular moments.

In [161]:
# prints the zeroth moment
print(mycirc[0])

# prints the last moment
print(mycirc[-1])

# prints a range of moments as a circuit diagram
print(mycirc[1 : 4])

X(0) and Z(1) and T(2)
Z(0) and Z(1) and H(2)
0: ───X^0.5───T───X^0.5───

1: ───H───────X───X───────

2: ───X───────H───T───────


We can also add two circuits together and multiply circuits.

In [182]:
circ1 = random_circuit(2, 2)
circ2 = random_circuit(2, 2)

print(f"Circuit 1 looks like:\n {circ1}\n")
print(f"Circuit 2 looks like:\n {circ2}\n")

print(f"Circuit 1 + Circuit 2:\n {circ1 + circ2}\n")

print(f"3 * Circuit 1:\n {3 * circ1}")

Circuit 1 looks like:
 0: ───Z───H───

1: ───Z───Y───

Circuit 2 looks like:
 0: ───Y───Y───

1: ───X───X───

Circuit 1 + Circuit 2:
 0: ───Z───H───Y───Y───

1: ───Z───Y───X───X───

3 * Circuit 1:
 0: ───Z───H───Z───H───Z───H───

1: ───Z───Y───Z───Y───Z───Y───
